In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import time

In [63]:
service = Service("C:\\Users\\luca_\\Desktop\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe")
service.start()
driver = webdriver.Chrome()
driver.maximize_window()

search_query = "Academia AJK Jundiaí Google reviews"
driver.get(f"https://www.google.com/search?q={search_query}")
time.sleep(2)
driver.execute_script("window.scrollTo(0, 500)")
time.sleep(2)

try:
    see_all_reviews_button = driver.find_element(By.LINK_TEXT, 'Ver todos os comentários do Google')
    see_all_reviews_button.click()
    time.sleep(3)
except Exception as e:
    print("Could not find the 'See all reviews' button.")
    driver.quit()

element_inside_popup = driver.find_element(By.CLASS_NAME, 'review-dialog-list')        
scroll_height = driver.execute_script("return arguments[0].scrollTop;", element_inside_popup)
i=0 #to speed up trial, to be removed later

while i<11:

    driver.execute_script("arguments[0].scrollTop += 400;", element_inside_popup)
    time.sleep(2)
    new_height = driver.execute_script("return arguments[0].scrollTop;", element_inside_popup)

    if scroll_height == new_height:
        print("Reached the bottom of the scrollable content.")
        break
    else:
        scroll_height = new_height 
        i +=1

page_source = driver.page_source

driver.quit()


soup = BeautifulSoup(page_source, 'html.parser')

reviews = soup.find_all('div', class_='jxjCjc')

with open('academia_ajk_reviews.txt', 'w', encoding='utf-8') as file:
    for review in reviews:
        file.write(review.text + "\n")
        file.write("\n")

print("Reviews have been saved to academia_ajk_reviews.txt")

Reviews have been saved to academia_ajk_reviews.txt


In [3]:
from transformers import pipeline
import os

sentiment_analyzer = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

def analyze_sentiment(text):
    return sentiment_analyzer(text)

def process_file(file_path):

    if not os.path.isfile(file_path):
        print(f"Error: The file '{file_path}' does not exist.")
        return

    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    entries = text.strip().split('\n\n')

    for i, entry in enumerate(entries, 1):
        entry = entry.split('atrás', 1)[1].strip()
        result = analyze_sentiment(entry)
        print(f"Entry {i}:")
        print(f"Text: {entry[:100]}...")  
        print(f"Sentiment Analysis: {result}")
        print("-" * 40)

file_path = 'C:\\Users\\luca_\\Desktop\\Py Training\\Introdução\\academia_ajk_reviews.txt'
process_file(file_path)


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


Entry 1:
Text: O esporte tem o poder de transformar a vida de uma pessoa e aqui não foi diferente nosso garoto dese...
Sentiment Analysis: [{'label': '5 stars', 'score': 0.6596231460571289}]
----------------------------------------
Entry 2:
Text: Excelente academia de artes marciais, ótimos professores, além de ensinar karatê, ainda ensinam e pr...
Sentiment Analysis: [{'label': '5 stars', 'score': 0.7892501950263977}]
----------------------------------------
Entry 3:
Text: Excelente academia para o aprendizado e a prática do Karatê enquanto arte marcial. O ensino é levado...
Sentiment Analysis: [{'label': '4 stars', 'score': 0.6396740674972534}]
----------------------------------------
Entry 4:
Text: NovaExcelente didática no ensino do Karatê.  Meu filho está  na escola desde os cinco anos e ama mui...
Sentiment Analysis: [{'label': '5 stars', 'score': 0.7308983206748962}]
----------------------------------------
Entry 5:
Text: A academia de Karatê AJK é, sem dúvida, a melhor de Jundi